**DOING THIS FOR DOCUMENT/CORPUS LEVEL**

**IMPORT FROM PDF AND NET. CREATE TEXT FILES**

In [ ]:
#after installing pdfminer.six
#move samples folder from my Data folder
#using pdfminer.high_level.extract_text(pdf_file, password='', page_numbers=None,
# maxpages=0, caching=True, codec='utf-8', laparams=None

In [ ]:
from pdfminer.high_level import extract_text

In [ ]:
import gensim
from gensim.summarization import summarize
from gensim.summarization import keywords
from gensim.parsing.preprocessing import strip_multiple_whitespaces
from gensim import corpora, models, similarities

In [ ]:
import os
directory = '/Users/lawrence/'
print (os.listdir(directory))

In [ ]:
for filename in os.listdir(directory):
    if filename.endswith(".pdf"):

        print (filename)
        text=extract_text(filename)
        text=strip_multiple_whitespaces(text)  #may wish to turn this off for readability
        ##create text file and open it for writing
        #"w" letter in our argument, which indicates write and will create a file if it does not exist in library
        #Plus sign indicates both read and write.
        #The available option beside "w" are, "r" for read, and "a" for append
        f= open(filename+".txt","w+") 
        f.write(filename+"\n"+text)
        f.close()

In [ ]:
#Add a text file from Internet

import requests

text = requests.get('https://archive.org/stream/ProjectManagementForTheOilAndGasIndustry/ProjectManagementForTheOilAndGasIndustry_djvu.txt').text
text=text[70000:300000] #have stripped the front text by inspection only, not by BS
text=strip_multiple_whitespaces(text)  #may wish to turn this off for readability
filename="PM_guidance_for_Energy_Projects"
f= open(filename+".txt","w+") 
f.write(filename+"\n"+text)
f.close()


**USE GENSIM TO CREATE SUMMARIES AND FIND KEY_WORDS** This is adequate for a first pass
This may be the place to start when re-running if you have already created PDFs

In [ ]:
#GENSIM summary, keywords
Corpus_of_Summaries =[]

for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        with open(os.path.join(directory, filename)) as f:
            
            content = f.read()
            
            summary=(filename+'Summary\n'+summarize(content, ratio=0.01)) #word_count=20
            key_words=keywords(content, ratio=0.007)
            
            print ('\n\nFile',filename)
            print ('\nSummary:',summary,"\n")
            print ('\nKeywords:',key_words)
            
            Corpus_of_Summaries.append(summary)
            
            #print (content[0:100]) # testing the content is coming through
            # print(repr(summary)) #alternate version showing line breaks etc
    
            f.close()

In [ ]:
print(Corpus_of_Summaries[2]) # Could equally do this w full content, but starting simple

**IDENTIFY TOKENS AND MAKE-UP DICTIONARY**

In [75]:
# remove common words and tokenize.# Here we can add in some odd words we find in the output, or use the NLTK list
stoplist = set('for a of the and to in \uf06e  • \uf0b7 \uf0b7 \uf06e uf09 \uf09f'.split())
Tokens_in_Corpus = [[word for word in summary.lower().split() if word not in stoplist]
         for summary in Corpus_of_Summaries]

In [ ]:
# remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)
for text in Tokens_in_Corpus:
    for token in text:
        frequency[token] += 1

Frequent_Tokens_in_Corpus= [[token for token in text if frequency[token] > 1] for text in Tokens_in_Corpus]

from pprint import pprint  # pretty-printer
pprint(Frequent_Tokens_in_Corpus[4:5]) #these slices of lists go up to before the higher number. 

In [ ]:
#create dictionary, then map from ids to dictionary
dictionary = corpora.Dictionary(Frequent_Tokens_in_Corpus)
print(dictionary,"\n\n")
print(dictionary.token2id)

**CREATE BAG OF WORDS MODEL**

In [ ]:
#ie. a list of a list. For each document, we have a list of word frequency for each dictionary item
BAG_OF_WORDS_MODEL = [dictionary.doc2bow(text) for text in Frequent_Tokens_in_Corpus]
for c in BAG_OF_WORDS_MODEL:
    print(c)

From Quick start tutorial. 
"Now that we have vectorized our corpus we can begin to transform it using models. We use model as an abstract term referring to a transformation from one document representation to another. In gensim documents are represented as vectors so a model can be thought of as a transformation between two vector spaces. The details of this transformation are learned from the training corpus."

**CREATE TF-IDF MODEL**
One simple example of a model is tf-idf. The tf-idf model transforms vectors from the bag-of-words representation to a vector space where the frequency counts are weighted according to the relative rarity of each word in the corpus.
Let's initialize the tf-idf model, training it on our corpus.

In [79]:
# train the model
TFIDF_MODEL= models.TfidfModel(BAG_OF_WORDS_MODEL)

**CREATE TOPIC MODEL via LSI** via CBOW AND TFIDF

In [112]:
# LSI APPLIED ON TOP ON TFIDF
#now applying an LSI to the first corpus, by working on top of its representation as a TFIDF
# here we have created a two dim LSI space, like Deerwesters 1990 example
#Presumably we could create one on top of just the CBOW too
lsi_from_TFIDF= models.LsiModel(TFIDF_APPLIED_TO_TRAINING_CORPUS, id2word=dictionary, num_topics=3) # initialize an LSI transformation

#It is correct how it has this odd double-barrelled structure: 
#model = LsiModel(common_corpus, id2word=common_dictionary)
# >>> vectorized_corpus = model[common_corpus]  # vectorize input copus in BoW format

In [ ]:
#inspect the topics
lsi_from_TFIDF.print_topics(num_topics=-1, num_words=20) #-1 means show all topics .In significance order. Remember also _ve Contribs

In [84]:
#Get a single topic as a formatted string with print_topic(topicno, topn=10)

In [85]:
#to get as array use lsi.get_topics()

In [86]:
# can Update model with new corpus using add_documents(corpus, chunksize=None, decay=None)
#can also save the LSI model

**CREATE TOPIC MODEL LSI - FROM CBOW DIRECTLY**

In [ ]:
lsi_from_CBOW = models.LsiModel(BAG_OF_WORDS_MODEL, id2word=dictionary, num_topics=3) # initialize an LSI transformation
TOPIC_MODEL_LSI_APPLIED_TO_CORPUS_FROM_CBOW = lsi_from_CBOW[BAG_OF_WORDS_MODEL] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi
lsi_from_CBOW.print_topics(num_topics=-1, num_words=20) #-1 means show all topics .In significance order. Remember also _ve Contribs

**FINDING VECTOR REPRESENTATION OF A WHOLE OLD OR NEW CORPUS**
To prepare for similarity queries, we need to enter all documents which we want to compare against subsequent queries. In our case, they are the same documents used for training LSI, converted to 3-D LSA space. But that’s only incidental, we might also be indexing a different corpus altogether.

   **REPRESENTATION OF OLD CORPUS: TFIDF**

In [ ]:
#now moved onto Topic and Transformations tutorial
#apply tfidf to the trained corpus
TFIDF_APPLIED_TO_TRAINING_CORPUS = TFIDF_MODEL[BAG_OF_WORDS_MODEL]
for doc in TFIDF_APPLIED_TO_TRAINING_CORPUS:
    print(doc)

   **REPRESENTATION OF OLD CORPUS: LSI FROM TFIDF**

In [ ]:
# create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi
TOPIC_MODEL_LSI_from_TFIDF_APPLIED_TO_CORPUS = lsi_from_TFIDF[TFIDF_APPLIED_TO_TRAINING_CORPUS] 

#particular documents aligned to particular topics
for doc in TOPIC_MODEL_LSI_from_TFIDF_APPLIED_TO_CORPUS: # both bow->tfidf and tfidf->lsi transformations are actually executed here, on the fly
    print(doc)

In [81]:
#OPTIONAL
#this model can now be applied to another corpus other than the training one, not just individaul documents
#i have not pulled in a second corpus but this is how you would do it. Note you pull in a corpus (processed as above), not just docs. 
#corpus2nd_tfidf = TFIDF_MODEL[corpus2nd]
# for doc in corpus2nd_tfidf:
#   print(doc)

   **REPRESENTATION OF OLD CORPUS: LSI FROM CBOW ONLY**

In [ ]:
TOPIC_MODEL_LSI_from_CBOW_APPLIED_TO_CORPUS = lsi_from_CBOW[BAG_OF_WORDS_MODEL] 

for doc in TOPIC_MODEL_LSI_from_CBOW_APPLIED_TO_CORPUS : # both bow->tfidf and tfidf->lsi transformations are actually executed here, on the fly
    print(doc)

**FINDING VECTOR REPRESENTATION OF A SINGLE NEW DOCUMENT**

In [87]:
# Up above, we had a CBOW representation of each document
#We can convert documents to that vector space,once tokenized

# eg.This is the announcement of the Sellafield partner programme. https://www.gov.uk/government/news/sellafield-ltd-awards-20-year-project-partnership
#Which ONR document is most relevant to this contract ?

In [ ]:
page = requests.get("https://www.gov.uk/government/news/sellafield-ltd-awards-20-year-project-partnership")
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')
new_doc=strip_multiple_whitespaces(soup.get_text())
print(new_doc)

#page = requests.get("https://www.gov.uk/government/news/nda-sets-out-its-grand-challenges")
#page.content[1:300]
# need to find which tag works well with this approach. p does not work well with this NEC text 
#soup.find_all('p')[5].get_text()

In [ ]:
new_doc=new_doc[0:5487]
print(new_doc)

In addition, we will be considering cosine similarity to determine the similarity of two vectors. Cosine similarity is a standard measure in Vector Space Modeling, but wherever the vectors represent probability distributions, different similarity measures may be more appropriate.

   **REPRESENTATION OF NEW DOCUMENT: CBOW ONLY**

In [ ]:
#convert tokenized documents to vector
new_vec_CBOW = dictionary.doc2bow(new_doc.lower().split())

In [ ]:
print(new_vec_CBOW)  # only those words that match up are given a dimension

   **REPRESENTATION OF NEW DOCUMENT: TFIDF**

In [117]:
#convert the query to LSI space (based on TFIDF) 
new_vec_TFIDF=TFIDF_MODEL[new_vec_CBOW]

   **REPRESENTATION OF NEW DOCUMENT: LSI via TFIDF

In [117]:
new_vec_lsi_fromTFIDF = lsi_from_TFIDF[new_vec_TFIDF]
print(new_vec_lsi_fromTFIDF)  

[(0, 0.39447736857014587), (1, 0.044582505706060725), (2, 0.07041520267680661)]


 **REPRESENTATION OF NEW DOCUMENT: LSI via CBOW ONLY**

In [ ]:
new_vec_lsi_fromCBOW = lsi_from_CBOW[new_vec_CBOW]
print(new_vec_lsi_fromCBOW)  

**COSINE SIMILARITY**

moved onto Similarity search tutorial.
Based on this new doc query,we would like to sort our corpus documents in decreasing order of relevance to this query. Unlike modern search engines, here we only concentrate on a single aspect of possible similarities—on apparent semantic relatedness of their texts (words). No hyperlinks, no random-walk static ranks, just a semantic extension overthe boolean keyword match:

   **LSI VIA TFIDF**

In [118]:
# LSI APPLIED ON TOP ON TFIDF
index = similarities.MatrixSimilarity(lsi_from_TFIDF[TFIDF_APPLIED_TO_TRAINING_CORPUS]) # transform corpus to LSI space and index it

In [119]:
sims = index[new_vec_lsi_fromTFIDF] # perform a similarity query against the corpus BASED ON LSI - TDFIDF
print(list(enumerate(sims))) # print (document_number, document_similarity) 2-tuples

[(0, 0.6954509), (1, 0.6393743), (2, 0.7355601), (3, 0.65083766), (4, 0.6242627), (5, 0.5353125), (6, 0.6523766), (7, 0.5703195), (8, 0.7608185), (9, 0.97505426), (10, 0.5026522), (11, 0.5055611), (12, 0.98621565), (13, 0.6932255), (14, 0.96844214), (15, 0.9984242)]


Cosine measure returns similarities in the range <-1, 1> (the greater, the more similar), so that the first document has a score of 0.99809301 etc.

With some standard Python magic we sort these similarities into descending order, and obtain the final answer to the query for Sellafield PPP:

In [120]:
sims = sorted(enumerate(sims), key=lambda item: -item[1])
print(sims) # print sorted (document number, similarity score) 2-tuples

[(15, 0.9984242), (12, 0.98621565), (9, 0.97505426), (14, 0.96844214), (8, 0.7608185), (2, 0.7355601), (0, 0.6954509), (13, 0.6932255), (6, 0.6523766), (3, 0.65083766), (1, 0.6393743), (4, 0.6242627), (7, 0.5703195), (5, 0.5353125), (11, 0.5055611), (10, 0.5026522)]


In [ ]:
#Most like
print (Corpus_of_Summaries[15])

In [ ]:
print (Corpus_of_Summaries[12])

In [ ]:
print (Corpus_of_Summaries[9])

In [ ]:
#Least like
print (Corpus_of_Summaries[11])

In [ ]:
print (Corpus_of_Summaries[10])

 **LSI VIA CBOW ONLY**

In [126]:
# LSI APPLIED ON TOP ON TFIDF
index = similarities.MatrixSimilarity(lsi_from_CBOW[TOPIC_MODEL_LSI_APPLIED_TO_CORPUS_FROM_CBOW])
sims = index[new_vec_lsi_fromCBOW] 
print(list(enumerate(sims))) 
sims = sorted(enumerate(sims), key=lambda item: -item[1])
print("\n",sims)

[(0, 0.7372657), (1, 0.7495415), (2, 0.74884546), (3, 0.7513679), (4, 0.7469633), (5, 0.7520852), (6, 0.74919164), (7, 0.7491072), (8, 0.75197256), (9, -0.73850036), (10, 0.7532815), (11, 0.75439507), (12, 0.7511035), (13, 0.7512372), (14, 0.744389), (15, 0.75025356)]

 [(11, 0.75439507), (10, 0.7532815), (5, 0.7520852), (8, 0.75197256), (3, 0.7513679), (13, 0.7512372), (12, 0.7511035), (15, 0.75025356), (1, 0.7495415), (6, 0.74919164), (7, 0.7491072), (2, 0.74884546), (4, 0.7469633), (14, 0.744389), (0, 0.7372657), (9, -0.73850036)]


   **TFIDF ONLY**

In [ ]:
#Now the same but with TFIDF model
index = similarities.MatrixSimilarity(TFIDF_APPLIED_TO_TRAINING_CORPUS)
new_vec_TFIDF = TFIDF_MODEL[new_vec_CBOW] # convert the query to LSI space (based on TFIDF)
print(new_vec_TFIDF)  

In [ ]:
sims = index[new_vec_TFIDF] 
print(list(enumerate(sims)))
sims = sorted(enumerate(sims), key=lambda item: -item[1])
print ("\n")
print (sims)

   **CBOW ONLY**

In [ ]:
#Now the same FROM CBOW DIRECTLY
#perform a similarity query against the corpus BASED ON CBOW

In [ ]:
#Now the same but with CBOW model
index = similarities.MatrixSimilarity(BAG_OF_WORDS_MODEL)
sims = index[new_vec_CBOW] 
print(list(enumerate(sims)))
sims = sorted(enumerate(sims), key=lambda item: -item[1])
print ("\n")
print (sims)


**Using keywords into Neo4j concurrence...**
